In [ ]:
import numpy as np
import pandas as pd
import time

from IPython import display as dsp

from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def specific_kind(dictionary, kind):
        """
        Returns all instances of some kind of objects in dictianry
        Input:
            dictionary
            kind (string) – kind of the objects to return
            
        Return:
            (list) – list of objects of this specific kind    
        """
        return [it for it in dictionary.values() if it.type == kind]

In [ ]:
class World:
    """
    World. A place where everythong belongs...
    """
    def __init__(self, size):
        """
        Initializes the world
        
        Input:
            size (1x2 array) – size of the world along x (0'th element) and y (1'st element)
        """
        
        # Size of the world
        self.size = size
        
        # Current time in the world
        self.time = 0
        
        x = np.linspace(0, size[0])
        y = np.linspace(0, size[1])
        
        # X and Y meshgrid
        self.X, self.Y = np.meshgrid(x, y)
        
        # Objects in the world
        self.objects = {}
   

    def add(self, type_, xy, name, *args, **kwargs):
        """
        Adds an object to the world
        
        Input:
            type_ - type of the object to add
        """
        
        object_ = type_(self, xy, name, *args, **kwargs)
        
        # You can't place two objects in one position
        if self.occupied(object_.xy):
            raise ValueError('This position is occupied')  
        # Each object should have it's unique name
        if object_.name in self.objects:
            raise ValueError('Such name is occupied')
        # If everything is good – add an object to the world
        else:
            self.objects[object_.name] = object_ 
            
    
    def occupied(self, xy):
        """
        Checks if this position is ocupied
        
        Input:
            xy (1x2 array) – coordinates in the world
            
        Output:
            (boolean) – True if occupied, False if not
        """
        for ind in range(len(self.objects)):
            if np.all(xy == list(self.objects.values())[ind].xy):
                return True
        return False
    
    
    def move(self, name, xy):
        """
        Moves object from one xy to another
        
        Input:
            name (string) – name of the object to move
            xy (1x2 numpy.ndarray) – coordinates of the object where it should be moved
        """
        # If this position is occupied – raise ValueError
        if self.occupied(xy):
            raise ValueError('This position is occupied')
        # If this position is free – move the object
        else:
            self.objects[name].xy = np.array(xy)
    
    
    def delete(self, name):
        """
        Delets object
        
        Input:
            name (string) – name of the object to delete
        """
        del self.objects[name]
    
    
    def plot(self):
        """
        Show the plot of the world
        """
        plt.figure(figsize=(10, 10))
        ax = plt.gca()
        ax.set_aspect('equal')
        ax.set_axisbelow(True)
        plt.grid()
        plt.xlim([-1, self.size[0] + 1])
        plt.ylim([-1, self.size[1] + 1])

        rect = patches.Rectangle((0, 0), self.size[0], self.size[1], linewidth=1, edgecolor='k', facecolor='none')
        ax.add_patch(rect)
        
        x_berries = [it.xy[0] for it in list(specific_kind(self.objects(), berry))]
        y_berries = [it.xy[1] for it in list(specific_kind(self.objects(), berry))]
        s_berries = [it.nutr_val * 10 for it in list(specific_kind(self.objects(), berry))]
        plt.scatter(x_berries, y_berries, s=s_berries, marker='s', color='rebeccapurple')
        
        x_bricks = [it.xy[0] for it in list(specific_kind(self.objects(), brick))]
        y_bricks = [it.xy[1] for it in list(specific_kind(self.objects(), brick))]
        s_bricks = [it.weight * 10 for it in list(specific_kind(self.objects(), brick))]
        plt.scatter(x_bricks, y_bricks, s=s_bricks, marker='s', color='chocolate')
        
        x_creatures = [it.xy[0] for it in list(specific_kind(self.objects(), creature))]
        y_creatures = [it.xy[1] for it in list(specific_kind(self.objects(), creature))]
        plt.scatter(x_creatures, y_creatures, marker='o', color='crimson')
    
    
    def get_environment(self, xy, size):
        """
        Returns two dicts with all objects in the proximity ('near') and in the circle of radius "size" centered at "xy"
        Input:
            xy (1x2 array) – center for the circle
            size (float) – radius of the circle
            
        Return:
            near (dict) – dict with all objects in the neares cells
            far (dict) – dicr with all objects in the circle of radius "size"
        """
        near = {}
        far = {}
        
        for name, object_ in self.objects.items():
            
            if np.linalg.norm(object_.xy - xy) == 1:
                near[name] = object_
            elif np.linalg.norm(object_.xy - xy) <= size:
                far[name] = object_
                
        return near, far

    
class obj:
    """
    An instance of an object. Nothing in particular
    """
    def __init__(self, world, xy, name):
        self.world = world
        self.xy = np.array(xy)
        self.name = name
        self.type = 'object'
    
    def __str__(self):
        return f"Object {self.name} placed at {self.xy}"
    
    def __repr__(self):
        return f"Object at {self.xy}"
     
        
class brick(obj):
    def __init__(self, world, xy, name, weight=1):
        super().__init__(world, xy, name)
        self.weight = weight
        self.type = 'brick'
        
    def __str__(self):
        return f"Brick {self.name} placed at {self.xy} with weight={self.weight}"
    
    def __repr__(self):
        return f"Brick at {self.xy}"

    
class berry(obj):
    def __init__(self, world, xy, name, nutr_val=1):
        super().__init__(world, xy, name)
        self.nutr_val = nutr_val
        self.type = 'berry'
        
    def __str__(self):
        return f"Berry {self.name} placed at {self.xy} with nutrition value={self.nutr_val}"
    
    def __repr__(self):
        return f"Berry at {self.xy}"


class creature(obj):
    def __init__(self, world, xy, name, hp=10, perception=3):
        super().__init__(world, xy, name)
        self.type = 'creature'
        self.hp = hp
        self.perception = perception
        self.see = None
        self.touch = None
     
    
    def __str__(self):
        return f"Creature {self.name} placed at {self.xy}"
    
    
    def __repr__(self):
        return f"Creature at {self.xy}"
    
    
    def decision(self):
        berries = specific_kind(self.see,'berry')
        berries_xy = [it.xy for it in berries]
        print(berries_xy)
        if len(berries) > 0:
            distance = berries_xy - self.xy
            ind_min = min(range(len(distance)), key=distance.__getitem__)
            target = berries[ind_min]
            print(target.xy)
        
        
        
    
    
    def step(self, direction):
        new_xy = self.xy + np.array(direction)
        if not self.world.occupied(new_xy):
            self.xy = new_xy
        else:
            return 0
    
    
    def interact(self, direction):
        to_interact = None
        
        for name, object_ in self.touch.items():
            if np.all((object_.xy - self.xy) == direction):
                to_interact = object_
            break
            
        if to_interact == None:
            return 0
            
        elif to_interact.type == 'berry':
            self.health = self.healts + to_interact.nutr_val
            
        elif to_interact.type == 'brick':
            self.world.move(to_interact.name, to_interact.xy + direction)
            self.xy = self.xy + direction
                 
    
    def observe(self):
        self.touch, self.see = self.world.get_environment(self.xy, self.perception)
        del self.see[self.name]    

In [ ]:
w = World([10, 10])
w.add(brick, [2, 2], 'brick')
w.add(berry, [2, 4], 'berry')
w.add(creature, [2, 3], 'Tom', perception=10)
w.add(obj, [3, 3], 'ass')

# Tom = w.objects['Tom']
# Tom.observe()
# Tom.touch
# Tom.interact([0,-1])
# w.objects
Tom.observe()
Tom.decision()


In [ ]:
x = np.linspace(0, 2*np.pi)
fig = plt.figure(1)
for ind in range(10):
    plt.plot(x, np.sin(ind * x))
    dsp.display(plt.gcf())
    time.sleep(0.5)
    plt.cla()
    dsp.clear_output(wait=True)
    

In [ ]:
len({'1':1})

In [ ]:
values = [3, 3, 2, 4]
min(range(len(values)), key=values.__getitem__)

In [ ]:
ass = {'1': 1}

In [ ]:
list(ass.values())[0]